Para desarrollar un ejemplo de MEDA y oMEDA necesitamos inicialmente tan sólo un modelo PCA que con su modelado y sus variables latentes, y partiendo de eso haciendo los pasos descritos para los distintos métodos vamos a ir analizando las conclusiones.


# PCA
La idea aqui es plantear una matriz no totalmente aleatoria sino una realción aritmética con un vector sí aleatorio tipo
x1=vector aleatorio
x2=2*x1 + 5
x3=(3/2)*x2 - 4
x4=x3
x5=x3+6

Pero en principio lo dejamos todo aleatorizado para el despliegue de los cálculos...

In [1]:
import numpy as np
from datetime import datetime
import time
from scipy.stats import norm
import numpy.linalg as linalg

import matplotlib.pyplot as plt
%matplotlib inline

from matplotlib import pyplot
from mpl_toolkits.mplot3d import Axes3D

In [2]:
#Generación e matriz aleatoria arbitraria
nobs=100
nvar=10
lv=3
np.random.seed(1)
calibration_matrix=np.random.random((nobs,nvar)) #matriz de 300x112

#Guardamos la matriz en un csv:
import csv
with open('calibracion.csv', 'wb') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(calibration_matrix)

#comprobar

calibration_matrix=(calibration_matrix*10).astype(int)
print "La matriz generada es: \n", calibration_matrix


La matriz generada es: 
[[4 7 0 3 1 0 1 3 3 5]
 [4 6 2 8 0 6 4 5 1 1]
 [8 9 3 6 8 8 0 0 1 8]
 [0 4 9 5 6 3 6 8 0 7]
 [9 7 2 7 1 4 9 2 2 1]
 [0 6 2 2 4 0 5 1 5 6]
 [1 4 6 4 0 5 6 5 9 5]
 [9 1 1 8 3 1 9 3 7 7]
 [8 6 7 3 2 8 4 9 6 6]
 [1 9 4 5 4 2 9 5 0 6]
 [3 5 8 3 9 6 0 9 6 9]
 [1 1 9 6 0 7 7 9 7 1]
 [0 0 0 2 8 5 5 8 1 2]
 [5 9 5 0 8 2 8 3 8 7]
 [5 1 0 1 0 1 2 7 5 0]
 [0 9 5 2 2 7 1 5 9 8]
 [2 4 6 8 1 0 0 4 6 5]
 [3 9 5 3 5 7 6 2 0 3]
 [6 2 7 0 2 8 1 6 5 9]
 [2 0 7 7 9 9 0 2 6 9]
 [9 5 9 6 3 4 6 5 9 9]
 [3 9 1 1 1 5 0 9 8 0]
 [1 3 1 8 3 9 5 8 8 9]
 [4 5 7 2 4 5 0 5 4 8]
 [3 8 5 1 7 6 0 4 6 9]
 [0 9 3 9 6 8 5 6 2 5]
 [7 8 7 6 8 3 6 4 3 4]
 [4 3 6 4 9 6 1 4 3 7]
 [8 9 6 2 2 8 5 8 5 7]
 [5 7 5 4 3 0 3 0 9 1]
 [8 8 6 5 1 4 3 2 5 3]
 [9 9 2 1 1 4 7 2 2 8]
 [4 6 2 1 5 4 1 6 5 6]
 [1 7 2 5 7 0 3 8 8 5]
 [8 9 8 8 0 6 7 6 7 0]
 [7 7 2 2 6 3 7 4 7 9]
 [3 1 9 5 9 6 9 5 5 1]
 [3 0 1 7 0 3 8 6 6 1]
 [4 5 9 2 2 2 7 4 0 5]
 [2 7 2 0 5 8 3 7 5 5]
 [9 8 0 7 4 9 2 2 0 4]
 [1 6 8 6 7 3 8 4 8 6]
 [1 0 0 0 

In [3]:
x=calibration_matrix
average = np.nanmean(calibration_matrix,axis=0)# array of M elements
average = average.reshape((1,average.shape[0]))# Matrix of 1xM elements
scale = np.nanstd(calibration_matrix,axis=0,ddof=1)

nanM = np.isnan(x)
anM = 1 - nanM

#TODO: to ask Pepe what is this :(
ind = np.nonzero(scale == 0)# # of zeroes in scale
dem = 2.0*np.sum(anM[:,ind],axis=0) - 1
scale[ind] = np.sqrt(np.ones((1,np.array(ind).size)) / dem)

scale = scale.reshape((1,scale.shape[0]))# Matrix of 1xM elements


In [4]:
xcs = x - np.dot(np.ones((calibration_matrix.shape[0],1)),average)
xcs = xcs / np.dot(np.ones((calibration_matrix.shape[0],1)),scale)
xcs

array([[ 0.00342694,  0.65818622, -1.59059737, -0.47361899, -1.21998703,
        -1.53287331, -1.30145744, -0.43137466, -0.63524814,  0.15065418],
       [ 0.00342694,  0.32743937, -0.8990333 ,  1.33408709, -1.57156831,
         0.50642599, -0.27399104,  0.29362477, -1.38259889, -1.25078002],
       [ 1.37420258,  1.3196799 , -0.55325126,  0.61100465,  1.24108191,
         1.18619243, -1.64394624, -1.51887381, -1.38259889,  1.20172982],
       [-1.3673487 , -0.33405431,  1.52144096,  0.24946344,  0.53791935,
        -0.51322366,  0.41098656,  1.38112392, -1.75627427,  0.85137127],
       [ 1.71689649,  0.65818622, -0.8990333 ,  0.97254587, -1.21998703,
        -0.17334044,  1.43845296, -0.79387438, -1.00892351, -1.25078002],
       [-1.3673487 ,  0.32743937, -0.8990333 , -0.83516021, -0.1652432 ,
        -1.53287331,  0.06849776, -1.1563741 ,  0.11210261,  0.50101272],
       [-1.02465479, -0.33405431,  0.48409485, -0.11207778, -1.57156831,
         0.16654278,  0.41098656,  0.29362477

In [5]:
data=xcs
pcs=lv

u, s, v = linalg.svd(data)


In [6]:
# Transform S from array to matrix with the corresponding dimensions
# in U and V
sdiag = np.diag(s)
s = np.zeros((u.shape[1], v.shape[0]))
s[:sdiag.shape[0], :sdiag.shape[1]] = sdiag
s

array([[12.38513857,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        , 11.5994812 ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        , 11.48083174,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , 10.6393332 ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        , 10.05131388,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         9.11592794,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  8.95364487,  0.        

In [7]:
t = np.dot(u, s)  #  ->u: Matriz de similaridad de fila-clase     
t

array([[ 1.03556953e+00, -1.02143230e+00, -1.91266113e+00,
         3.03504193e-01,  1.11713455e+00,  9.22749627e-01,
         1.79006469e-01,  9.48674146e-01, -4.81993495e-01,
         4.49223137e-01],
       [-2.37172145e-01,  1.11345865e+00, -1.57863705e+00,
        -1.44487656e+00,  4.71683406e-01,  1.61290941e+00,
         5.23826928e-01, -5.49300036e-02,  1.44188414e-01,
        -2.46820654e-01],
       [-1.74701965e+00,  2.83027577e-01, -1.27474093e+00,
         2.46497081e+00,  1.46278439e+00,  2.80928529e-01,
         7.21570145e-01, -7.95540911e-01,  1.12640728e+00,
         3.84422164e-01],
       [-9.41223976e-01, -5.92440122e-02,  1.58129058e+00,
        -5.05036217e-01,  3.82169366e-01,  5.74375190e-02,
         1.71526101e+00,  7.83950716e-02, -1.94868638e+00,
        -5.06753987e-01],
       [ 4.09905223e-01,  2.58307353e+00, -1.71549253e+00,
        -6.43491642e-01,  7.94304538e-01, -1.12496426e-01,
        -3.47908110e-01,  8.39602302e-01,  4.97913802e-01,
        -1.

In [8]:
p = v.T         #->v: Matriz de similaridad de columna-clase
p

array([[-0.35607259,  0.39650613, -0.32523454,  0.23313408, -0.11902516,
        -0.26657   , -0.22099945,  0.44065194,  0.21905103,  0.42384124],
       [ 0.20353532,  0.02882917, -0.54854869,  0.09177414,  0.06383353,
        -0.4476509 ,  0.54725721, -0.37227476,  0.05570088,  0.05259764],
       [-0.12408174,  0.19503943,  0.3709543 ,  0.24800976, -0.63452841,
        -0.11707625,  0.48577639,  0.15619338, -0.12266059, -0.23599485],
       [-0.07049137,  0.61974938, -0.08412052,  0.05479061, -0.08358178,
         0.43969453, -0.06212938, -0.50971103, -0.32150372,  0.18307424],
       [-0.09063752,  0.00663464,  0.62374182,  0.28075127,  0.33701007,
        -0.22804189,  0.11299154, -0.30299503,  0.23118088,  0.44751437],
       [-0.59745753, -0.0190359 , -0.05327919, -0.17679265, -0.12140648,
        -0.02249598, -0.09803013, -0.40565871,  0.52614136, -0.37729112],
       [ 0.1590723 ,  0.49072759,  0.21218906, -0.33295164,  0.17596549,
        -0.57823369, -0.26353526, -0.00498268

In [9]:
 
scoresMatrix = t[:, :pcs]  # (Las dos pc's)
scoresMatrix

array([[ 1.03556953, -1.0214323 , -1.91266113],
       [-0.23717214,  1.11345865, -1.57863705],
       [-1.74701965,  0.28302758, -1.27474093],
       [-0.94122398, -0.05924401,  1.58129058],
       [ 0.40990522,  2.58307353, -1.71549253],
       [ 1.9088744 , -1.0953539 , -0.10754765],
       [ 0.78984594, -0.64924255, -0.17258963],
       [ 0.29686811,  1.60521752, -0.44797212],
       [-1.79209465, -0.35879344, -0.89752059],
       [ 0.43692119,  0.6262924 , -0.30066454],
       [-1.74848679, -1.93850525,  1.14967752],
       [-0.1647673 ,  0.40182941,  0.92551912],
       [-0.43045962, -1.10652076,  1.746392  ],
       [ 1.07829102, -0.62663061,  0.31404535],
       [ 0.9206166 , -1.16837246, -0.7903605 ],
       [ 0.17864035, -2.41684634, -1.0970991 ],
       [ 0.93039063, -0.27786383, -0.56623557],
       [-0.20993337,  0.52645594, -0.37635776],
       [-2.08679974, -1.85322259, -0.23459749],
       [-1.74851578, -0.78040717,  1.74038899],
       [-0.75315668,  0.69097553, -0.379

In [10]:

loadingsMatrix = p[:, :pcs]
loadingsMatrix



array([[-0.35607259,  0.39650613, -0.32523454],
       [ 0.20353532,  0.02882917, -0.54854869],
       [-0.12408174,  0.19503943,  0.3709543 ],
       [-0.07049137,  0.61974938, -0.08412052],
       [-0.09063752,  0.00663464,  0.62374182],
       [-0.59745753, -0.0190359 , -0.05327919],
       [ 0.1590723 ,  0.49072759,  0.21218906],
       [-0.34789178, -0.20975205,  0.04872933],
       [ 0.37303281, -0.21135398,  0.01487809],
       [-0.4009533 , -0.29989284, -0.0955031 ]])

In [11]:
model = np.dot(scoresMatrix, loadingsMatrix.T) #Matriz de 300x112
model

#Con este codigo podremos ver que la media de cada una de las columnas es 0
#average2 = np.nanmean(model,axis=0)# array of M elements
#average2 = average2.reshape((1,average2.shape[0]))
#average2

array([[-1.51678635e-01,  1.23051568e+00, -1.03722470e+00,
        -5.45136697e-01, -1.29364503e+00, -4.97359890e-01,
        -7.42360356e-01, -2.39221294e-01,  5.73728448e-01,
         7.37702784e-02],
       [ 1.03937098e+00,  8.49786455e-01, -3.39005125e-01,
         8.39579671e-01, -9.55777864e-01,  2.04613104e-01,
         1.73707841e-01, -2.27965909e-01, -3.47294014e-01,
        -8.80586000e-02],
       [ 1.14887777e+00,  3.51836704e-01, -2.00895849e-01,
         4.05787847e-01, -6.34885914e-01,  1.10629953e+00,
        -4.09499074e-01,  4.86290890e-01, -7.30480368e-01,
         7.37337048e-01],
       [-2.02636869e-01, -1.06069515e+00,  6.91820322e-01,
        -1.03387264e-01,  1.07123422e+00,  4.79219231e-01,
         1.56737232e-01,  4.16925857e-01, -3.15059383e-01,
         2.44135566e-01],
       [ 1.43618591e+00,  1.09892922e+00, -1.83429890e-01,
         1.71627157e+00, -1.09003947e+00, -2.02672036e-01,
         9.68781256e-01, -7.68002407e-01, -4.18558026e-01,
        -7.

In [12]:
data=xcs
residualsMatrix = data - model #
residualsMatrix

array([[ 1.55105574e-01, -5.72329467e-01, -5.53372670e-01,
         7.15177047e-02,  7.36579980e-02, -1.03551342e+00,
        -5.59097085e-01, -1.92153369e-01, -1.20897659e+00,
         7.68838976e-02],
       [-1.03594404e+00, -5.22347081e-01, -5.60028172e-01,
         4.94507415e-01, -6.15790444e-01,  3.01812889e-01,
        -4.47698881e-01,  5.21590680e-01, -1.03530488e+00,
        -1.16272142e+00],
       [ 2.25324810e-01,  9.67843199e-01, -3.52355411e-01,
         2.05216808e-01,  1.87596782e+00,  7.98928964e-02,
        -1.23444717e+00, -2.00516470e+00, -6.52118522e-01,
         4.64392775e-01],
       [-1.16471183e+00,  7.26640837e-01,  8.29620643e-01,
         3.52850703e-01, -5.33314861e-01, -9.92442888e-01,
         2.54249328e-01,  9.64198064e-01, -1.44121488e+00,
         6.07235708e-01],
       [ 2.80710579e-01, -4.40742999e-01, -7.15603407e-01,
        -7.43725700e-01, -1.29947556e-01,  2.93315956e-02,
         4.69671704e-01, -2.58719726e-02, -5.90365488e-01,
        -4.

In [13]:
eigengvaluesMatrix = s
eigengvaluesMatrix

array([[12.38513857,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        , 11.5994812 ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        , 11.48083174,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , 10.6393332 ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        , 10.05131388,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         9.11592794,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  8.95364487,  0.        

# MEDA

In [20]:
P = loadingsMatrix
T = scoresMatrix
e = residualsMatrix

x = calibration_matrix

## Procedemos

In [21]:
#Contruimos una matriz Xm NxM full zeros excepto la mth columna que contendrá la m-ésima columna de
#la matriz x (Calibración) original
xm = np.zeros((nobs, nvar))

xm.shape

(100, 10)

In [22]:
#Ahora debemos sustituir la columna m-esima de 0's por la columna de la matriz original con todas
#las columnas (nvar)

In [25]:
#Para cada una de las columnas generamos una matriz xm tal que sustituimos la mésima columna de
#la mtriz de ceros con su respectiva de la matriz de calibración x
x1=xm
x1[:,0] = x[:,0]
x1



array([[4., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [4., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [8., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [9., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [9., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [8., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [3., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [5., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [5., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [2., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [3., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [6., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [2., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [9., 0., 0., 

In [ ]:
#Teniendo x1 definido el segundo paso es aplicar KDR

### KDR

In [17]:
#Debemos aplicar la metodología del método de datos perdidos (Missing Data Method), 
#en particula de Known Data Regression (KDR) para las M matrices obtenidas de la iteración anterior.

#Definimos la matriz indicador de datos perdidos